In [ ]:
import numpy as np 
import pandas as pd
import nltk
nltk.download('stopwords')
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

MAX_FEATURES = 20000
MAX_WORD = 100
EMBEDDING_SIZE = 300



In [ ]:
train_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
sample_submission = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

In [ ]:
def text_prepare(text):
    """Tokenization and Preprocessing."""
    
    # Make everything lowercase
    text = text.lower()
    
    # Remove misspelled words or words not found in GoogleNews embeddings (determined in data exploration and preprocessing)
    text = re.sub("doesnt", "does not", text)
    text = re.sub("dont", "do not", text)
    text = re.sub("isnt", "is not", text)
    text = re.sub("wasnt", "was not", text)
    text = re.sub("didnt", "did not", text)
    text = re.sub("behaviour", "behavior", text)
    text = re.sub("colour", "color", text)
    
    # Replace symbols,newline characters and remove stopwords. Then tokenize sentence 
    replace_by_space_re = re.compile('[/(){}\[\]\|@,;]')
    good_symbols_re = re.compile('[^0-9a-z #+_]')
    stopwords_set = set(stopwords.words('english'))
    text = re.sub("\n", " ", text)
    text = replace_by_space_re.sub(' ', text)
    text = good_symbols_re.sub('', text)
    text = ' '.join([x for x in nltk_tokenize(text) if x and x not in stopwords_set])
    
    return text.strip()


def nltk_tokenize(text):
    """Used to split a sentence into tokens"""
    
    tokens = word_tokenize(text)
    return tokens


In [ ]:
# Clean comments in our training set and test set
train_cleaned_x = train_x['comment_text'].map(lambda x: text_prepare(x))
test_cleaned_x = test_x['comment_text'].map(lambda x: text_prepare(x))

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Use keras tokenizer to find 20,000 most important words and prepare for them to be fed into embedding layer
tokenizer = Tokenizer(MAX_FEATURES)
tokenizer.fit_on_texts(list(train_cleaned_x))
train_tokenized_x = tokenizer.texts_to_sequences(train_cleaned_x)
test_tokenized_x = tokenizer.texts_to_sequences(test_cleaned_x)

# Pad input so that they are all of the same length
padded_train_x = pad_sequences(train_tokenized_x, maxlen = MAX_WORD)
padded_test_x = pad_sequences(test_tokenized_x, MAX_WORD)

In [ ]:
"""
IGNORE: USED IN DATA EXPLORATION


def create_dict(sentences):
    dict = {}
    for sentence in sentences:
        for word in sentence:
            try:
                dict[word] += 1
            except:
                dict[word] = 1
    return dict

def unk(dict, word2vec):
    unk_token = {}
    for word in dict:
        try:
            word2vec[word]
        except:
            unk_token[word] = dict[word]
    return unk_token
"""      

In [ ]:
import gensim

# Use gensim to load GoogleNews pre-trained word embeddings
INPUT_FILE = "../input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(INPUT_FILE, binary=True)

In [ ]:
# Convert gensim object into a python dictionary
dict_size = len(tokenizer.word_index)
embedding_index = {}
for word in word2vec.wv.vocab:
    embedding_index[word] = word2vec.word_vec(word)


In [ ]:
# Create a matrix of embeddings and pick the default values from the distribution of matrix weights
embeddings = np.stack(list(embedding_index.values()))
embeddings_mean = embeddings.mean()
embeddings_std_dev = embeddings.std()
embedding_matrix = np.random.normal(embeddings_mean, embeddings_std_dev, (len(tokenizer.word_index), EMBEDDING_SIZE))

In [ ]:
# For words from the source text (fed into the tokenizer), change the weights in the matrix to match their embedding values
for word, index in tokenizer.word_index.items():
    if word in embedding_index:
        embedding_matrix[index - 1] = embedding_index[word]

In [ ]:
# Import model and layers used
from keras.models import Model
from keras.layers import  Embedding, Input, concatenate, SpatialDropout1D, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, Bidirectional, GRU, Dense

# Use keras to define model
emb = Input(shape=(None,))
x = Embedding(dict_size,embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False)(emb)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(256, return_sequences = True, dropout = 0.2, recurrent_dropout = 0.2))(x)
x = Conv1D(64, kernel_size = 4)(x)
x = concatenate([GlobalMaxPooling1D()(x), GlobalAveragePooling1D()(x)])
x = Dense(6, activation = "sigmoid")(x)
model = Model(emb, x)
model.compile(loss = 'binary_crossentropy', optimizer= 'adam', metrics = ['accuracy'])

In [ ]:
# Fit model on our padded data
classifier = model.fit(padded_train_x, train_y.values, batch_size = 1024, epochs = 5, validation_split = 0.1)

In [ ]:
# Predict outcomes for our test data and save as a csv
prediction = model.predict(padded_test_x, batch_size = 1024)
sample_submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = prediction
sample_submission.to_csv('submission.csv', index = False)
